In [4]:
from sklearn.metrics import r2_score, mean_squared_log_error, max_error

import pandas as pd
import numpy as np
import datetime
from time import perf_counter

from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.base import ForecastingHorizon
from sktime.performance_metrics.forecasting import sMAPE, smape_loss
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.arima import ARIMA, AutoARIMA

from tabulate import tabulate

import warnings
warnings.filterwarnings('ignore')

date_parser = lambda ts: datetime.strptime(ts, "%y-%m-%dT%H:%M:%SZ")
df = pd.read_csv('aviasales_data_t.csv')
df['requested_at'] = pd.to_datetime(df['requested_at'], format="%y-%m-%dT%H:%M:%SZ", errors="coerce")
df['departure_at'] = pd.to_datetime(df['departure_at'], format="%Y-%m-%dT%H:%M:%SZ", errors="coerce")
df['expires_at'] = pd.to_datetime(df['expires_at'], format="%Y-%m-%dT%H:%M:%SZ", errors="coerce")
df['price'] = pd.to_numeric(df["price"], errors="coerce")
df = df.dropna()
df = df.drop_duplicates(subset=['departure_at', 'expires_at', 'airline', 'flight_number', 'price'], keep='first')


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


# Profile prediction func and test prediction quality
def ml_model_test(orgn : str, dest : str,
                  current_date : datetime.datetime, flight_date : datetime.datetime,
                  predictor_f, target_f):
    print(orgn, dest)

    tic = perf_counter()
    predictions = predictor_f(orgn, dest, current_date, flight_date)
    tac = perf_counter()
    elapsed_time = tac - tic
    
    targets = target_f(orgn, dest, current_date, flight_date)
    print(f"Elapsed time: {elapsed_time:0.4f} seconds")
    print(predictions[1], targets[1])
    
    r2 = r2_score(targets[1], predictions[1])
    msle = mean_squared_log_error(targets[1], predictions[1])
    max_err = max_error(targets[1], predictions[1])
    mape = mean_absolute_percentage_error(targets[1], predictions[1])
    
    print("R2:", r2)
    print("MSLE:", msle)
    print("Max error:", max_err)
    print("MAPE:", mape)
    
    return (str(predictor_f), orgn, dest, elapsed_time, r2, msle, max_err, mape)

# INSERT YOUR FUNCTION
def predict_queried_prices(orgn, dest, current_date, flight_date):
    days = []
    predictions = []
    delta = flight_date - current_date
    for i in range(delta.days + 1):
        days.append((flight_date - datetime.timedelta(days=i)).strftime("%Y-%m-%d"))
        predictions.append(predict_queried_prices_for(orgn, dest, current_date, flight_date, i))
    return days, predictions

def predict_queried_prices_for(orgn, dest, current_date, flight_date, n_days):
    
    orgn_dest = df[(df["origin"] == orgn) & (df["destination"] == dest) & (df["departure_at"] <= flight_date) 
                   & (df["requested_at"] >= df["departure_at"] - datetime.timedelta(days=n_days + 1))
                   & (df["requested_at"] <= df["departure_at"] - datetime.timedelta(days=n_days))] 
    
    orgn_dest_day_min = orgn_dest.resample('D', on='requested_at')['price'].min()
  
    orgn_dest_day_min = orgn_dest_day_min.fillna(orgn_dest_day_min.mean())  # not very smart
    
    y = orgn_dest_day_min
    y_train, y_test = temporal_train_test_split(y)
    
    forecaster = AutoARIMA()
    forecaster.fit(y_train)
    fh = ForecastingHorizon(y_test.index, is_relative=False)
    y_pred = forecaster.predict(fh)
    return round(y_pred[-1])

# INSERT YOUR FUNCTION
def show_real_prices(orgn, dest, current_date, flight_date, delta_days=7):
    
    orgn_dest = df[(df["origin"] == orgn) & (df["destination"] == dest) & (df["departure_at"] <= flight_date + datetime.timedelta(days=1))
                   & (df["departure_at"] >= flight_date)]
    
    days = []
    prices = []
    for i in range(delta_days + 1):
        orgn_dest_day = orgn_dest[(orgn_dest["requested_at"] >= orgn_dest["departure_at"] - datetime.timedelta(days=i + 1))
                           & (orgn_dest["requested_at"] <= orgn_dest["departure_at"] - datetime.timedelta(days=i))]
        if orgn_dest_day.empty:
            orgn_dest_day = orgn_dest[(orgn_dest["expires_at"] >= orgn_dest["departure_at"] - datetime.timedelta(days=i + 1))
                               & (orgn_dest["expires_at"] <= orgn_dest["departure_at"] - datetime.timedelta(days=i))]
        
        prices.append(round(orgn_dest_day["price"].min()))
        days.append((flight_date - datetime.timedelta(days=i)).strftime("%Y-%m-%d"))
        
    return days, prices



# JUST AN EXAMPLE

i = "MOW"
ml_test_results = []
for i in ['MOW', 'LED']: # NaNs in others , 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD'
    for j in ['MOW', 'LED', 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD']: # , 'SGC'
        if i != j:
            ml_test_results.append(ml_model_test(i, j, datetime.datetime(2021,4,20), datetime.datetime(2021,4,27),
                                             predict_queried_prices, show_real_prices))
ml_test_results.append(ml_model_test("MOW", "SGC", datetime.datetime(2021,4,20), datetime.datetime(2021,4,27),
                                             predict_queried_prices, show_real_prices))            
            
ml_test_df = pd.DataFrame(ml_test_results)
ml_test_df.columns = ['predictor_func', 'origin', 'destination', 'elapsed_time', 'r2_score', 'msle_score', 'max_error', 'mape']
print(tabulate(ml_test_df, headers = 'keys', tablefmt = 'psql'))

    

MOW LED
Elapsed time: 3.9345 seconds
[3930, 2693, 2636, 2617, 2617, 2682, 2848, 2848] [3221, 2498, 2595, 2531, 2890, 2655, 2890, 3148]
R2: -0.32481120463089264
MSLE: 0.00834292436089073
Max error: 709
MAPE: 7.030291731898397
MOW KZN
Elapsed time: 6.0650 seconds
[5349, 4556, 4366, 4359, 4293, 4282, 4444, 4483] [4498, 4172, 4248, 4226, 4172, 4172, 4248, 4248]
R2: -11.745940830760036
MSLE: 0.005737168359761949
Max error: 851
MAPE: 6.216445506304873
MOW CEK
Elapsed time: 3.1468 seconds
[6591, 6146, 6174, 6151, 6388, 5975, 6490, 6212] [5460, 4998, 4998, 5777, 6148, 6148, 5635, 5817]
R2: -2.58624915757484
MSLE: 0.01915975915956455
Max error: 1176
MAPE: 12.79599160761019
MOW SVX
Elapsed time: 3.6844 seconds
[5714, 4341, 4361, 4287, 4282, 4095, 4280, 4279] [4058, 3658, 3953, 3658, 3658, 3658, 3658, 4058]
R2: -17.615820238919877
MSLE: 0.030769018373809326
Max error: 1656
MAPE: 17.306366145876105
MOW AER
Elapsed time: 3.1762 seconds
[6354, 5891, 6062, 6003, 5763, 6042, 6221, 6059] [3955, 4123, 5

In [23]:
from sklearn.metrics import r2_score, mean_squared_log_error, max_error

import pandas as pd
import numpy as np
import datetime
from time import perf_counter

from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.base import ForecastingHorizon
from sktime.performance_metrics.forecasting import sMAPE, smape_loss
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.arima import ARIMA, AutoARIMA

from tabulate import tabulate

import warnings
warnings.filterwarnings('ignore')

date_parser = lambda ts: datetime.strptime(ts, "%y-%m-%dT%H:%M:%SZ")
df = pd.read_csv('aviasales_data_t.csv')
df['requested_at'] = pd.to_datetime(df['requested_at'], format="%y-%m-%dT%H:%M:%SZ", errors="coerce")
df['departure_at'] = pd.to_datetime(df['departure_at'], format="%Y-%m-%dT%H:%M:%SZ", errors="coerce")
df['expires_at'] = pd.to_datetime(df['expires_at'], format="%Y-%m-%dT%H:%M:%SZ", errors="coerce")
df['price'] = pd.to_numeric(df["price"], errors="coerce")
df = df.dropna()
df = df.drop_duplicates(subset=['departure_at', 'expires_at', 'airline', 'flight_number', 'price'], keep='first')


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


# Profile prediction func and test prediction quality
def ml_model_test(orgn : str, dest : str,
                  current_date : datetime.datetime, flight_date : datetime.datetime,
                  predictor_f, target_f, n, m):
    # print(orgn, dest)

    tic = perf_counter()
    predictions = predictor_f(orgn, dest, current_date, flight_date, n, m)
    tac = perf_counter()
    elapsed_time = tac - tic
    
    targets = target_f(orgn, dest, current_date, flight_date)
    # print(f"Elapsed time: {elapsed_time:0.4f} seconds")
    # print(predictions[1], targets[1])
    
    r2 = r2_score(targets[1], predictions[1])
    msle = mean_squared_log_error(targets[1], predictions[1])
    max_err = max_error(targets[1], predictions[1])
    mape = mean_absolute_percentage_error(targets[1], predictions[1])
    
    # print("R2:", r2)
    # print("MSLE:", msle)
    # print("Max error:", max_err)
    # print("MAPE:", mape)
    
    return (str(predictor_f), orgn, dest, elapsed_time, r2, msle, max_err, mape)

# INSERT YOUR FUNCTION
# def predict_queried_prices(orgn, dest, current_date, flight_date):
#     days = []
#     predictions = []
#     delta = flight_date - current_date
#     for i in range(delta.days + 1):
#         days.append((flight_date - datetime.timedelta(days=i)).strftime("%Y-%m-%d"))
#         predictions.append(predict_queried_prices_for(orgn, dest, current_date, flight_date, i))
#     return days, predictions

# def predict_queried_prices_for(orgn, dest, current_date, flight_date, n_days):
    
#     orgn_dest = df[(df["origin"] == orgn) & (df["destination"] == dest) & (df["departure_at"] <= flight_date) 
#                    & (df["requested_at"] >= df["departure_at"] - datetime.timedelta(days=n_days + 1))
#                    & (df["requested_at"] <= df["departure_at"] - datetime.timedelta(days=n_days))] 
    
#     orgn_dest_day_min = orgn_dest.resample('D', on='requested_at')['price'].min()
  
#     orgn_dest_day_min = orgn_dest_day_min.fillna(orgn_dest_day_min.mean())  # not very smart
    
#     y = orgn_dest_day_min
#     y_train, y_test = temporal_train_test_split(y)
    
#     forecaster = AutoARIMA()
#     forecaster.fit(y_train)
#     fh = ForecastingHorizon(y_test.index, is_relative=False)
#     y_pred = forecaster.predict(fh)
#     return round(y_pred[-1])

# # INSERT YOUR FUNCTION
# def show_real_prices(orgn, dest, current_date, flight_date, delta_days=7):
    
#     orgn_dest = df[(df["origin"] == orgn) & (df["destination"] == dest) & (df["departure_at"] <= flight_date + datetime.timedelta(days=1))
#                    & (df["departure_at"] >= flight_date)]
    
#     days = []
#     prices = []
#     for i in range(delta_days + 1):
#         orgn_dest_day = orgn_dest[(orgn_dest["requested_at"] >= orgn_dest["departure_at"] - datetime.timedelta(days=i + 1))
#                            & (orgn_dest["requested_at"] <= orgn_dest["departure_at"] - datetime.timedelta(days=i))]
#         if orgn_dest_day.empty:
#             orgn_dest_day = orgn_dest[(orgn_dest["expires_at"] >= orgn_dest["departure_at"] - datetime.timedelta(days=i + 1))
#                                & (orgn_dest["expires_at"] <= orgn_dest["departure_at"] - datetime.timedelta(days=i))]
        
#         prices.append(round(orgn_dest_day["price"].min()))
#         days.append((flight_date - datetime.timedelta(days=i)).strftime("%Y-%m-%d"))
        
#     return days, prices

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import xgboost as xgb


def get_int(s):
    try:
        return int(s)
    except:
        return None


def init_df() -> pd.DataFrame:
    df = pd.read_csv('aviasales_data_t.csv')
    df['departure_time'] = pd.to_datetime(df['departure_at'], errors='coerce')
    df['requested_time'] = pd.to_datetime('20' + df['requested_at'], errors='coerce')
    df['before_flight'] = (df['departure_time'] - df['requested_time']).apply(
        lambda x: x.total_seconds() / 3600)
    df['requested_date'] = df['requested_time'].apply(lambda x: x.date())
    df['departure_date'] = df['departure_time'].apply(lambda x: x.date())
    df = pd.DataFrame(df[['price', 'before_flight', 'origin', 'destination', 'departure_date', 'requested_date']])
    df['price'] = df['price'].apply(lambda x: get_int(x))
    df = df.groupby(['departure_date', 'requested_date', 'origin', 'destination']).min()

    df = df.reset_index()

    return df


data = init_df()


def prepare_df(origin, destination) -> pd.DataFrame:
    df = data.loc[data['origin'] == origin]
    df = df.loc[df['destination'] == destination]
    df.drop(['origin', 'destination'], axis=1)

    return df


def create_only_date_train_features(df):
    rows = [create_date_features(df.iloc[i]) for i in range(len(df))]
    curr_df = pd.DataFrame(rows)
    return curr_df


def create_date_features(old_row):
    row = {}
    req_date = pd.to_datetime(old_row['requested_date'])
    row['req_dayofweek'] = req_date.dayofweek
    row['req_quarter'] = req_date.quarter
    row['req_month'] = req_date.month
    row['req_year'] = req_date.year
    row['req_dayofyear'] = req_date.dayofyear
    row['req_dayofmonth'] = req_date.day
    row['req_weekofyear'] = req_date.weekofyear

    dep_date = pd.to_datetime(old_row['departure_date'])
    row['dep_dayofweek'] = dep_date.dayofweek
    row['dep_quarter'] = dep_date.quarter
    row['dep_month'] = dep_date.month
    row['dep_year'] = dep_date.year
    row['dep_dayofyear'] = dep_date.dayofyear
    row['dep_dayofmonth'] = dep_date.day
    row['dep_weekofyear'] = dep_date.weekofyear

    row['before_flight'] = old_row['before_flight']
    return row


def predict(origin, destination, current_date, flight_date, n, m):
    df = prepare_df(origin, destination)
    df = df[df['departure_date'] < flight_date.date()]
    y_train = df['price']
    X_train = create_only_date_train_features(df)
    model = xgb.XGBRegressor(n_estimators=n, max_depth=m).fit(X_train, y_train)

    rows = []
    days = []
    delta = flight_date - current_date
    for i in range(delta.days + 1):
        req_date = (flight_date - timedelta(days=i)).date()
        fl_date = flight_date.date()
        before_flight = timedelta(days=i).total_seconds() / 3600
        days.append(str(req_date))
        rows.append({'departure_date': fl_date, 'requested_date': req_date, 'before_flight': before_flight})

    X_test = create_only_date_train_features(pd.DataFrame(rows))
    y_pred = model.predict(X_test)
    return days, list(y_pred)


def show_real_prices(origin, destination, current_date, flight_date, delta_days=7):
    df = prepare_df(origin, destination)
    days = []
    prices = []
    for i in range(delta_days + 1):
        req_date = (flight_date - timedelta(days=i)).date()
        fl_date = flight_date.date()
        days.append(str(req_date))
        price = None
        try:
            cur_df = df.loc[df['departure_date'] == fl_date]
            price = cur_df.loc[cur_df['requested_date'] == req_date]['price'].iloc[0]
        except:
            pass
        prices.append(price)

    return days, list(prices)


# JUST AN EXAMPLE

# i = "MOW"
# ml_test_results = []
# for i in ['MOW', 'LED']: # NaNs in others , 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD'
#     for j in ['MOW', 'LED', 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD']: # , 'SGC'
#         if i != j:
#             ml_test_results.append(ml_model_test(i, j, datetime(2021,4,20), datetime(2021,4,27),
#                                              predict, show_real_prices))
# ml_test_results.append(ml_model_test("MOW", "SGC", datetime(2021,4,20), datetime(2021,4,27),
#                                              predict, show_real_prices))            
            
# ml_test_df = pd.DataFrame(ml_test_results)
# ml_test_df.columns = ['predictor_func', 'origin', 'destination', 'elapsed_time', 'r2_score', 'msle_score', 'max_error', 'mape']
# print(tabulate(ml_test_df, headers = 'keys', tablefmt = 'psql'))

    

In [2]:
! pip install sktime


     |████████████████████████████████| 5.7MB 4.6MB/s 
     |████████████████████████████████| 22.3MB 1.6MB/s 
     |████████████████████████████████| 9.5MB 184kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [3]:
! pip install delayed


     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 92kB 3.9MB/s 


In [4]:
! pip install pmdarima


     |████████████████████████████████| 1.5MB 4.0MB/s 


In [21]:
from tqdm.notebook import tqdm

n_estimators = [50, 100, 150, 200, 300, 400, 500]
max_depth = [5, 10, 15, 20, 30, 40, 50]

best_n = 0
best_m = 0
best_mape = np.inf

for n in tqdm(n_estimators):
  for m in max_depth:
    cur_mape = 0
    for i in ['MOW', 'LED']: # NaNs in others , 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD'
        for j in ['MOW', 'LED', 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD']: # , 'SGC'
            if i != j:
                _, mape = ml_model_test(i, j, datetime(2021,4,20), datetime(2021,4,27),
                                                predict, show_real_prices, n, m)
                cur_mape += mape
    if best_mape > cur_mape:
       best_mape = cur_mape
       best_n = n
       best_m = m      
            

[02:25:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:25:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:25:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:25:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:25:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:25:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:25:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:25:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [22]:
print(best_n, best_m)

50 10


In [25]:
i = "MOW"
ml_test_results = []
for i in ['MOW', 'LED']: # NaNs in others , 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD'
    for j in ['MOW', 'LED', 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD']: # , 'SGC'
        if i != j:
            ml_test_results.append(ml_model_test(i, j, datetime(2021,4,20), datetime(2021,4,27),
                                             predict, show_real_prices, 50, 10))
ml_test_results.append(ml_model_test("MOW", "SGC", datetime(2021,4,20), datetime(2021,4,27),
                                             predict, show_real_prices, 50, 10))            
            
ml_test_df = pd.DataFrame(ml_test_results)
ml_test_df.columns = ['predictor_func', 'origin', 'destination', 'elapsed_time', 'r2_score', 'msle_score', 'max_error', 'mape']
print(tabulate(ml_test_df, headers = 'keys', tablefmt = 'psql'))

[02:40:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:40:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:40:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:40:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:40:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:40:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:40:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:40:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [26]:
i = "MOW"
ml_test_results = []
for i in ['MOW', 'LED']: # NaNs in others , 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD'
    for j in ['MOW', 'LED', 'KZN', 'CEK', 'SVX', 'AER', 'KRR', 'KGD']: # , 'SGC'
        if i != j:
            ml_test_results.append(ml_model_test(i, j, datetime(2021,4,20), datetime(2021,4,27),
                                             predict, show_real_prices, 200, 20))
ml_test_results.append(ml_model_test("MOW", "SGC", datetime(2021,4,20), datetime(2021,4,27),
                                             predict, show_real_prices, 200, 20))            
            
ml_test_df = pd.DataFrame(ml_test_results)
ml_test_df.columns = ['predictor_func', 'origin', 'destination', 'elapsed_time', 'r2_score', 'msle_score', 'max_error', 'mape']
print(tabulate(ml_test_df, headers = 'keys', tablefmt = 'psql'))

[02:41:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:41:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:41:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:41:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:41:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:41:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:41:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:41:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re